In [ ]:
from google.colab import files


In [ ]:
uploaded = files.upload()

Saving WA_Fn-UseC_-Telco-Customer-Churn.csv to WA_Fn-UseC_-Telco-Customer-Churn.csv


In [ ]:
import pandas as pd
import io
df = pd.read_csv(io.BytesIO(uploaded['WA_Fn-UseC_-Telco-Customer-Churn.csv']))


In [ ]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [ ]:
df.isnull().sum()

,0
customerID,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler


In [ ]:

df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')


df['TotalCharges'] = df['TotalCharges'].fillna(0)



In [ ]:
binary_cols = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling',
               'Churn', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
               'TechSupport', 'StreamingTV', 'StreamingMovies', 'MultipleLines']

le = LabelEncoder()
for col in binary_cols:
    df[col] = le.fit_transform(df[col])


In [ ]:
multi_cat_cols = ['InternetService', 'Contract', 'PaymentMethod', 'gender']

df = pd.get_dummies(df, columns=multi_cat_cols, drop_first=True)


In [ ]:
scaler = StandardScaler()

numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges', 'SeniorCitizen']
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, RocCurveDisplay


In [ ]:

y = df['Churn']


X = df.drop(['customerID', 'Churn'], axis=1)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

In [ ]:

y_pred = model.predict(X_test)


y_prob = model.predict_proba(X_test)[:, 1]


In [ ]:

cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)


print("\nClassification Report:\n", classification_report(y_test, y_pred))




Confusion Matrix:
 [[926 109]
 [165 209]]

Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.89      0.87      1035
           1       0.66      0.56      0.60       374

    accuracy                           0.81      1409
   macro avg       0.75      0.73      0.74      1409
weighted avg       0.80      0.81      0.80      1409



In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.81


In [ ]:

churn_risk = pd.DataFrame({'CustomerID': df.loc[y_test.index, 'customerID'],
                           'Churn_Probability': y_prob})
churn_risk.sort_values(by='Churn_Probability', ascending=False).head(10)


,CustomerID,Churn_Probability
3380,5178-LMXOP,0.855705
6866,0295-PPHDO,0.833544
2631,6861-XWTWQ,0.827386
6365,8884-ADFVN,0.826093
3727,9057-SIHCH,0.821318
2797,6023-YEBUP,0.819956
4585,1069-XAIEM,0.819752
6894,1400-MMYXY,0.818417
3346,2545-EBUPK,0.816579
4039,8161-QYMTT,0.810508


In [ ]:

y_prob = model.predict_proba(X_test)[:, 1]


In [ ]:
import pandas as pd


y_pred_class = (y_prob >= 0.5).astype(int)


In [ ]:
customer_classification = pd.DataFrame({
    'CustomerID': df.loc[y_test.index, 'customerID'],
    'Churn_Probability': y_prob,
    'Prediction': y_pred_class
})

customer_classification['Prediction_Label'] = customer_classification['Prediction'].map({
    0: 'Likely to Stay',
    1: 'Likely to Churn'
})


customer_classification.sort_values(by='Churn_Probability', ascending=False).head(10)


,CustomerID,Churn_Probability,Prediction,Prediction_Label
3380,5178-LMXOP,0.855705,1,Likely to Churn
6866,0295-PPHDO,0.833544,1,Likely to Churn
2631,6861-XWTWQ,0.827386,1,Likely to Churn
6365,8884-ADFVN,0.826093,1,Likely to Churn
3727,9057-SIHCH,0.821318,1,Likely to Churn
2797,6023-YEBUP,0.819956,1,Likely to Churn
4585,1069-XAIEM,0.819752,1,Likely to Churn
6894,1400-MMYXY,0.818417,1,Likely to Churn
3346,2545-EBUPK,0.816579,1,Likely to Churn
4039,8161-QYMTT,0.810508,1,Likely to Churn


In [ ]:
customer_classification['Prediction_Label'].value_counts()


,count
Prediction_Label,
Likely to Stay,1091
Likely to Churn,318


In [ ]:
import pandas as pd
import numpy as np


new_customers = pd.DataFrame({
    'SeniorCitizen': [0, 1],
    'tenure': [12, 1],
    'MonthlyCharges': [70, 90],
    'TotalCharges': [840, 90],
    'Partner': ['Yes', 'No'],
    'Dependents': ['No', 'No'],
    'PhoneService': ['Yes', 'Yes'],
    'MultipleLines': ['No', 'Yes'],
    'InternetService': ['Fiber optic', 'DSL'],
    'OnlineSecurity': ['No', 'Yes'],
    'OnlineBackup': ['Yes', 'No'],
    'DeviceProtection': ['No', 'Yes'],
    'TechSupport': ['No', 'No'],
    'StreamingTV': ['Yes', 'No'],
    'StreamingMovies': ['No', 'Yes'],
    'Contract': ['Month-to-month', 'Two year'],
    'PaperlessBilling': ['Yes', 'Yes'],
    'PaymentMethod': ['Electronic check', 'Mailed check']
})


binary_cols = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling',
               'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
               'TechSupport', 'StreamingTV', 'StreamingMovies', 'MultipleLines']

for col in binary_cols:
    new_customers[col] = le.transform(new_customers[col])


multi_cat_cols = ['InternetService', 'Contract', 'PaymentMethod']
new_customers = pd.get_dummies(new_customers, columns=multi_cat_cols, drop_first=True)


for col in X_train.columns:
    if col not in new_customers.columns:
        new_customers[col] = 0
new_customers = new_customers[X_train.columns]


numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges', 'SeniorCitizen']
new_customers[numeric_cols] = scaler.transform(new_customers[numeric_cols])


new_prob = model.predict_proba(new_customers)[:,1]
new_pred_class = (new_prob >= 0.5).astype(int)

new_customers['Churn_Probability'] = new_prob
new_customers['Prediction_Label'] = pd.Series(new_pred_class).map({0:'Likely to Stay', 1:'Likely to Churn'})



print(new_customers[['Churn_Probability','Prediction_Label']])


   Churn_Probability Prediction_Label
0           0.591191  Likely to Churn
1           0.197129   Likely to Stay
